In [36]:
import requests

In [44]:
data = {
    "action": "user_login",
    "user_login": "411317572@qq.com",
    "user_pass": "Aa411317572",
    "remember_me": "1"
}
res = requests.post("http://www.jobbole.com/wp-admin/admin-ajax.php", data=data)
print(res.status_code)
print(eval(res.content))
print(res.headers)

200
{'jb_result': -1, 'jb_msg': '用户名或密码错误'}
{'Server': 'nginx', 'Date': 'Mon, 27 May 2019 07:00:03 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Keep-Alive': 'timeout=2', 'Vary': 'Accept-Encoding', 'X-Powered-By': 'PHP/5.3.3', 'X-Robots-Tag': 'noindex', 'X-Content-Type-Options': 'nosniff', 'Expires': 'Wed, 11 Jan 1984 05:00:00 GMT', 'Cache-Control': 'no-cache, must-revalidate, max-age=0', 'Pragma': 'no-cache', 'X-Frame-Options': 'SAMEORIGIN', 'Content-Encoding': 'gzip'}


In [38]:
data = {
    "action": "user_login",
    "user_login": "411317572@qq.com",
    "user_pass": "Aa411317572",
    "remember_me": "1"
}
res = requests.get("http://www.jobbole.com/wp-admin/admin-ajax.php", data=data)
print(res.status_code)
print(eval(res.content))
print(res.headers)

200
0
{'Server': 'nginx', 'Date': 'Mon, 27 May 2019 06:39:12 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Keep-Alive': 'timeout=2', 'Vary': 'Accept-Encoding', 'X-Powered-By': 'PHP/5.3.3', 'Content-Encoding': 'gzip'}


## Zaker 爬去

In [59]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [60]:
url = "http://www.myzaker.com/channel/8"

In [61]:
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser', from_encoding='utf-8')
result = []

In [63]:
tags_a = soup.find_all('a')
for i in tags:
    result.append(urljoin(url, i["href"]))

In [64]:
tags_link = soup.find_all('link')
for i in tags_link:
    result.append(urljoin(url, i["href"]))
    print(urljoin(url, i["href"]))

http://zkres.myzaker.com
http://zkres1.myzaker.com
http://zkres2.myzaker.com
http://zkres.myzaker.com/static/zaker_web2/css/index.min.css?v=20170726
http://zkres.myzaker.com/static/zaker_web2/css/normalize.min.css?v=20170122
http://zkres.myzaker.com/static/zaker_web2/css/base.min.css?v=20170726
http://zkres.myzaker.com/static/zaker_web2/css/hotSearch.min.css?v=20170726
http://zkres.myzaker.com/static/zaker_web2/css/location.min.css?v=20171027


In [65]:
print(len(result))

244


In [67]:
for i in result:
#     print(i)
    pass

## 数据抓取

简单部分

`app`中的数据比`web`端数据更容易抓取, 反爬虫没有那么强, 大部分都是`http/https`协议, 返回的数据大部分为`json`

困难部分

1. 可能需要反编译, 分析出加密算法并抓取到信息
2. 可能加固, 需要脱壳, 然后反编译, 分析出加密算法并抓取信息
3. 需要破解通过各式各样的签名, 证书, 设备绑定等方法, 找到隐藏加密算法

掌握技术

1. `python`爬虫经验
2. `app`逆向
3. `java`开发基础
4. `app`脱壳
5. `android`开发基础
6. 破解加密算法

## 基于Docker打造的多任务抓取系统

![](./pictures/2019-05-29_11-22.png)

## GSLB 爬取站内所有url

In [1]:
import os
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse


class Spider_Gslb(object):

    def __init__(self, url, excluded_url):
        self.url = url  # 入口
        self.wait_url = set()  # 等待爬取的url
        self.already_done = set(excluded_url)  # 已经爬取过的url
        self.error_url = set()  # status code 不是200的url
        self.time_out_url = set()  # 超时的url
        self.page_size = 0
        self.url_domain_name = None

    def init_start_url_data(self):
        self.url_domain_name = urlparse(self.url).netloc

        res = requests.get(self.url)
        soup = BeautifulSoup(res.text, 'html.parser')
        tags = soup.find_all('a')
        for i in tags:
            self.wait_url.add(urljoin(self.url, i.get("href")))

    def web_page_size(self, res):
        try:
            page_size = res.headers["Content-Length"]
        except:
            with open("temp.html", "w") as fw:
                fw.write(res.text)
            page_size = os.path.getsize("temp.html")
        return page_size

    def crawl(self):
        while True:  # 循环不断从待爬取的集合中取出数据
            try:
                url_one = self.wait_url.pop()
            except:  # 当集合为空时, 爬取完毕, 跳出循环
                print("爬虫结束\n")
                break
            if url_one not in self.already_done:
                if urlparse(url_one).netloc not in self.url_domain_name:  # 通过域名判断是否是第三方连接, 如果是不爬取
                    self.already_done.add(url_one)
                else:
                    try:
                        res = requests.get(url_one, timeout=5)
                    except:
                        self.time_out_url.add(url_one)
                    else:
                        if res.status_code != 200:
                            self.error_url.add(url_one)
                        else:
                            print("链接: {}, 文件大小: {}Byte, Referer: {}".format(res.url, self.web_page_size(res), res.headers.get("Referer", None)))
                            print(res.headers)
                            soup = BeautifulSoup(res.text, 'html.parser')
                            tags = soup.find_all('a')
                            for i in tags:
                                temp_url = urljoin(self.url, i.get("href"))  # 判断第二子页是否存在爬取过的url
                                if temp_url not in self.already_done:
                                    self.wait_url.add(temp_url)
                            self.already_done.add(url_one)
            # print("剩余:{}, 已完成:{}, 超时:{}, 状态码非200:{}".format(len(self.wait_url), len(self.already_done),
            #                                                 len(self.time_out_url), len(self.error_url)))
            time.sleep(0.5)

    def print_data(self):
        print("总的url数量:", len(self.already_done), ", 如下:")
        for i in self.already_done:
            print(i)
        print("状态码非200数量:", len(self.error_url), ", 如下:")
        for i in self.error_url:
            print(i)
        print("请求超时url数量:", len(self.time_out_url), ", 如下:")
        for i in self.time_out_url:
            print(i)

    # 程序启动入口
    def start_app(self):
        self.init_start_url_data()
        self.crawl()
        self.print_data()


start_url = "http://www.cloudtopspeed.com/"
already_done = [start_url + "index.html"]
Spider_Gslb(url=start_url, excluded_url=already_done).start_app()


链接: http://www.cloudtopspeed.com/, 文件大小: 7762Byte, Referer: None
{'Server': 'CACHE/CLOUDSPEED_1.0.4V', 'Date': 'Tue, 18 Jun 2019 02:31:33 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'X-Powered-By': 'Express', 'Set-Cookie': 'clientIp=117.28.112.195; Max-Age=2592000; Path=/; Expires=Thu, 18 Jul 2019 02:31:33 GMT, location=%7B%22cityName%22%3A%22%22%2C%22cityCode%22%3A%22%22%2C%22province%22%3A%22%22%7D; Max-Age=2592000; Path=/; Expires=Thu, 18 Jul 2019 02:31:33 GMT', 'ETag': '"1e52-ow59uUu9J00BUY49VuPqGjb8thI"', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'Cache-Status': 'MISS'}
链接: http://www.cloudtopspeed.com/products/data.html, 文件大小: 8326Byte, Referer: None
{'Server': 'CACHE/CLOUDSPEED_1.0.4V', 'Date': 'Tue, 18 Jun 2019 02:31:36 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'X-Powered-By': 'Express', 'Set-Cookie': 'clientIp=117.28.112.195; Max-Ag

In [ ]:
start_url = "http://103.118.36.248"    
already_done = [start_url+"/", start_url+"/auth/login", start_url+"/auth/register"]
Spider_Gslb(url=start_url, excluded_url=already_done).start_app()

In [69]:
res = requests.head("http://www.cloudtopspeed.com/", timeout=5)

In [70]:
print(type(res.headers))

<class 'requests.structures.CaseInsensitiveDict'>


In [71]:
print(res.headers["Content-Length"])

172
